# 标普500指数预测 - 一次关于LSTM的探究
Notes包含如下部分
1. 导包
2. 导数据集
3. 构建LSTM神经网络
4. 训练网络

In [92]:
# SP500 Index prediction by LSTM model

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import pandas
import numpy as np

# Randomness function
torch.manual_seed(1)

In [110]:
# 排除日期数据
sp500 = pandas.read_csv("data.csv",usecols=[1,2,3,4,5,6,7,8])
# 填充NaN值为0，此步可优化
sp500.fillna(0, inplace=True)
# 去除columns空格否则模型报错
# sp500 = sp500.rename(columns={'Consumer Price Index': 'Consumer_Price_Index', 'Long Interest Rate': 'Long_Interest_Rate', 'Real Price': 'Real_Price', 'Real Dividend': 'Real_Dividen', 'Real Earnings': 'Real_Earnings'})
# sp500.columns = sp500.columns.str.stripe()

sp500.columns

Index(['SP500', 'Dividend', 'Earnings', 'Consumer Price Index',
       'Long Interest Rate', 'Real Price', 'Real Dividend', 'Real Earnings'],
      dtype='object')

In [139]:
# 测试验证数据集划分
train_dataset = sp500[:1000]
test_dataset = sp500[1000:]

# 数据缩放 - 标准化
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
train_dataset_normalized = scaler.fit_transform(train_dataset)
test_dataset_normalized = scaler.fit_transform(test_dataset)

# 创建tensor
train_dataset_tensor = map(
    torch.tensor, (train_dataset_normalized)
)

In [140]:
# LSTM模型构建，下行注释为参考教程网站
# https://www.jessicayung.com/lstms-for-time-series-in-pytorch/

# 定义LSTM神经网络模型类
class LSTM(nn.Module):

    def __init__(self, input_dim, hidden_dim, batch_size, output_dim=1, num_layers=2):
        super(LSTM, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        self.num_layers = num_layers

        # Define the LSTM layer
        self.lstm = nn.LSTM(self.input_dim, self.hidden_dim, self.num_layers)

        # Define the output layer
        self.linear = nn.Linear(self.hidden_dim, output_dim)

    def init_hidden(self):
        # This is what we'll initialise our hidden state as
        return (torch.zeros(self.num_layers, self.batch_size, self.hidden_dim),
                torch.zeros(self.num_layers, self.batch_size, self.hidden_dim))

    def forward(self, input):
        # Forward pass through LSTM layer
        # shape of lstm_out: [input_size, batch_size, hidden_dim]
        # shape of self.hidden: (a, b), where a and b both 
        # have shape (num_layers, batch_size, hidden_dim).
        # lstm_out, self.hidden = self.lstm(input.view(len(input), self.batch_size, -1))
        lstm_out, self.hidden = self.lstm(input.view(len(input), self.batch_size))
        
        # Only take the output from the final timetep
        # Can pass on the entirety of lstm_out to the next layer if it is a seq2seq prediction
        y_pred = self.linear(lstm_out[-1].view(self.batch_size, -1))
        return y_pred.view(-1)


# 实例化模型
# 模型数据维度，隐藏维度，批处理大小
input_dim = 8
hidden_dim = 8
output_dim = 1
num_train = 128
num_layers=8
model = LSTM(input_dim, hidden_dim, batch_size=num_train, output_dim=output_dim, num_layers=num_layers)

In [142]:
# 模型训练

num_epochs=8

loss_fn = torch.nn.MSELoss(size_average=False)

# lr=learning_rate, lr is set to default
optimiser = torch.optim.Adam(model.parameters())

# Train model
hist = np.zeros(num_epochs)

for t in range(num_epochs):
    # Clear stored gradient
    model.zero_grad()
    
    # Initialise hidden state
    # Don't do this if you want your LSTM to be stateful
    model.hidden = model.init_hidden()
    
    # Forward pass
    y_pred = model(train_dataset_tensor)

    loss = loss_fn(y_pred, y_train)
    if t % 100 == 0:
        print("Epoch ", t, "MSE: ", loss.item())
    hist[t] = loss.item()

    # Zero out gradient, else they will accumulate between epochs
    optimiser.zero_grad()

    # Backward pass
    loss.backward()

    # Update parameters
    optimiser.step()

AttributeError: 'map' object has no attribute 'view'